In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import tensorflow as tf
import os
#import cv2 
import numpy as np 
import math 

print(tf.__version__)

In [ ]:
X_celeb = np.load('/media/ml/Data Disk/upscaling/X_train64.npy')
Y_celeb = np.load('/media/ml/Data Disk/upscaling/Y_train64.npy')

X_tiny = np.load('/media/ml/Data Disk/upscaling/tinyface/X_train.npy')
Y_tiny = np.load('/media/ml/Data Disk/upscaling/tinyface/Y_train.npy')

#testx = np.load('')
#testy = np.load('')
train_size_celeb = 202599
num_classes_celeb = 10177
num_classes_tiny = 2570
file_size = 64

X_celeb = (X_celeb[:,:,:,:]/255).astype('float16')
X_tiny = (X_tiny[:,:,:,:]/255).astype('float16')


def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y
  
Y_celeb = convert_to_one_hot(Y_celeb, num_classes_celeb).T  
Y_tiny = convert_to_one_hot(Y_tiny, num_classes_tiny).T  


print(X_celeb.shape, Y_celeb.shape)
print(X_tiny.shape, Y_tiny.shape)

#print(X_test.shape, Y_test.shape)

In [ ]:

def srcnn(X, num_op):

    gen1 = tf.layers.conv2d(X, 64, 9, 1, padding = 'valid')
    A1 = tf.nn.relu(gen1)    
    P1 = tf.nn.max_pool(A1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
    
    gen2 = tf.layers.conv2d(P1, 32, 3, 1, padding = 'SAME')
    A2 = tf.nn.relu(gen2)    
    P2 = tf.nn.max_pool(A2, ksize = [1,4,4,1], strides = [1,4,4,1], padding = 'SAME')

    res_in = P2
    rc1 = tf.layers.conv2d(res_in, 16, 3, 1, padding = 'SAME')
    r2 = tf.layers.conv2d(rc1, 16, 1, 1, padding = 'SAME')
    r3 = tf.layers.conv2d(r2, 16, 1, 1, padding = 'SAME')
    r4 = tf.layers.conv2d(r3, 16, 1, 1, padding = 'SAME')
    r5 = tf.layers.conv2d(r4, 16, 1, 1, padding = 'SAME')
    r6 = tf.layers.conv2d(r5, 16, 1, 1, padding = 'SAME')
    r7 = tf.layers.conv2d(r6, 16, 1, 1, padding = 'SAME')
    res_out = tf.keras.layers.concatenate([res_in, r7])
    res_out = tf.nn.relu(res_out)   

    res_in = res_out
    rc1 = tf.layers.conv2d(res_in, 16, 3, 1, padding = 'SAME')
    r2 = tf.layers.conv2d(rc1, 16, 1, 1, padding = 'SAME')
    r3 = tf.layers.conv2d(r2, 16, 1, 1, padding = 'SAME')
    r4 = tf.layers.conv2d(r3, 16, 1, 1, padding = 'SAME')
    r5 = tf.layers.conv2d(r4, 16, 1, 1, padding = 'SAME')
    r6 = tf.layers.conv2d(r5, 16, 1, 1, padding = 'SAME')
    r7 = tf.layers.conv2d(r6, 16, 1, 1, padding = 'SAME')
    res_out = tf.keras.layers.concatenate([res_in, r7])
    res_out = tf.nn.relu(res_out)   

    res_in = res_out
    rc1 = tf.layers.conv2d(res_in, 16, 3, 1, padding = 'SAME')
    r2 = tf.layers.conv2d(rc1, 16, 1, 1, padding = 'SAME')
    r3 = tf.layers.conv2d(r2, 16, 1, 1, padding = 'SAME')
    r4 = tf.layers.conv2d(r3, 16, 1, 1, padding = 'SAME')
    r5 = tf.layers.conv2d(r4, 16, 1, 1, padding = 'SAME')
    r6 = tf.layers.conv2d(r5, 16, 1, 1, padding = 'SAME')
    r7 = tf.layers.conv2d(r6, 16, 1, 1, padding = 'SAME')
    res_out = tf.keras.layers.concatenate([res_in, r7])
    res_out = tf.nn.relu(res_out)    
    
    gen3 = tf.layers.conv2d(res_out, 3, 5, 1, padding = 'SAME')
    
    P3 = tf.nn.relu(gen3)  
    
    
    P_fl = tf.contrib.layers.flatten(P3)
    fc = tf.contrib.layers.fully_connected(P_fl, num_op, activation_fn = None)
    
    return P3, fc

def inception(X, num_op):

    A1 = tf.nn.relu(X)    
    
    gen11 = tf.layers.conv2d(A1, 64, 1, 1, padding = 'SAME')

    gen21 = tf.layers.conv2d(A1, 32, 1, 1, padding = 'SAME')
    gen22 = tf.layers.conv2d(gen21, 16, 3, 1, padding = 'SAME')

    gen31 = tf.layers.conv2d(A1, 16, 1, 1, padding = 'SAME')
    gen32 = tf.layers.conv2d(gen31, 32, 3, 1, padding = 'SAME')
    gen33 = tf.layers.conv2d(gen32, 64, 3, 1, padding = 'SAME')

    l4 = tf.keras.layers.concatenate([gen11, gen22, gen33])
    l5 = tf.keras.layers.concatenate([l4, A1])
    
    A6 = tf.nn.relu(l5)    
    
    P_fl = tf.contrib.layers.flatten(A6)
    fc = tf.contrib.layers.fully_connected(P_fl, num_op, activation_fn = None)
    
    return fc

In [ ]:
tf.reset_default_graph()
imported_meta = tf.train.import_meta_graph("/media/ml/Data Disk/upscaling/model/model.ckpt.meta")

(m, n_H0, n_W0, n_C0) = X_celeb.shape  
(m1, n_H1, n_W1, n_C1) = X_tiny.shape             
n_y = Y_celeb.shape[1]       
n_y1 = Y_tiny.shape[1]                            

costs = []                                                            
t1 = 0
t2 = 0
learning_rate = 0.0001
num_epochs = 500

X_c = tf.placeholder(tf.float32, [None, n_H0, n_W0, n_C0], name = 'X_c')
Y_c = tf.placeholder(tf.float32, [None, n_y], name = 'Y_c')
X_t = tf.placeholder(tf.float32, [None, n_H1, n_W1, n_C1], name = 'X_t')
Y_t = tf.placeholder(tf.float32, [None, n_y1], name = 'Y_t')

Pc, Z1c = srcnn(X_c, num_classes_celeb)
Z2c = inception(Pc, num_classes_celeb)

Pt, Z1t = srcnn(X_t, num_classes_tiny)
Z2t = inception(Pt, num_classes_tiny)



init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    imported_meta.restore(sess, tf.train.latest_checkpoint('/media/ml/Data Disk/upscaling/model'))
    print("restored")
    predict_op = tf.argmax(Z2c, 1)
    correct_prediction = tf.equal(predict_op, tf.argmax(Y_c, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    train_accuracy = accuracy.eval({X_c: X_celeb, Y_c: Y_celeb, X_t: X_tiny, Y_t: Y_tiny})
    #, X_t: X_tiny, Y_t:Y_tiny
    print("Train Accuracy:", train_accuracy)

ModuleNotFoundError: No module named 'models'